In [1]:
from pycaret.classification import *
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import requests
from bs4 import BeautifulSoup
import statistics
import math

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [3]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))

In [4]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [5]:
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [18]:
today = f"{datetime.now():%Y%m%d}"
# today = '20220830'
print(today)
game_data = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links()
days_data = connect_links(game_data, all_odds)

20220905


In [92]:
days_data

{'2022-09-05 13:23:00': [['monza-vs-atalanta',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919124/matchfacts/monza-vs-atalanta',
   'https://www.oddschecker.com/football/italy/serie-a/monza-v-atalanta/winner'],
  ['salernitana-vs-empoli',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919125/matchfacts/salernitana-vs-empoli',
   'https://www.oddschecker.com/football/italy/serie-a/salernitana-v-empoli/winner']],
 '2022-09-05 13:53:00': [['agf-aarhus-vs-nordsj%C3%A6lland',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887479/matchfacts/agf-aarhus-vs-nordsj%C3%A6lland',
   'https://www.oddschecker.com/football/denmark/superligaen/agf-aarhus-v-fc-nordsjaelland/winner']],
 '2022-09-05 14:53:00': [['boavista-vs-pacos-de-ferreira',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937427/matchfacts/boavista-vs-pacos-de-ferreira',
   'https://www.oddschecker.com/football/portugal/primeira-liga/boavista-v-pacos-ferreira/winner']],
 '2022-09-05

In [84]:
days_data = {'2022-09-03 08:23:00': [['everton-vs-liverpool',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900986/matchfacts/everton-vs-liverpool',
   'https://www.oddschecker.com/football/english/premier-league/everton-v-liverpool/winner'],
  ['celtic-vs-rangers',
   'SCOTLAND - PREMIERSHIP',
   'https://www.fotmob.com/match/3902849/matchfacts/celtic-vs-rangers',
   'https://www.oddschecker.com/football/scottish/premiership/celtic-v-rangers/winner']],
 '2022-09-03 08:53:00': [['mallorca-vs-girona',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917967/matchfacts/mallorca-vs-girona',
   'https://www.oddschecker.com/football/spain/la-liga-primera/mallorca-v-girona/winner']],
 '2022-09-03 09:53:00': [['fiorentina-vs-juventus',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919120/matchfacts/fiorentina-vs-juventus',
   'https://www.oddschecker.com/football/italy/serie-a/fiorentina-v-juventus/winner']],
 '2022-09-03 10:23:00': [['bayer-leverkusen-vs-sc-freiburg',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903581/matchfacts/bayer-leverkusen-vs-sc-freiburg',
   'https://www.oddschecker.com/football/germany/bundesliga/bayer-leverkusen-v-sc-freiburg/winner'],
  ['vfl-bochum-vs-werder-bremen',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903586/matchfacts/vfl-bochum-vs-werder-bremen',
   'https://www.oddschecker.com/football/germany/bundesliga/bochum-v-werder-bremen/winner'],
  ['union-berlin-vs-bayern-m%C3%BCnchen',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903582/matchfacts/union-berlin-vs-bayern-m%C3%BCnchen',
   'https://www.oddschecker.com/football/germany/bundesliga/union-berlin-v-bayern-munich/winner'],
  ['vfb-stuttgart-vs-schalke-04',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903588/matchfacts/vfb-stuttgart-vs-schalke-04',
   'https://www.oddschecker.com/football/germany/bundesliga/vfb-stuttgart-v-schalke/winner'],
  ['wolfsburg-vs-1.-fc-k%C3%B6ln',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903585/matchfacts/wolfsburg-vs-1.-fc-k%C3%B6ln',
   'https://www.oddschecker.com/football/germany/bundesliga/wolfsburg-v-cologne/winner']],
 '2022-09-03 10:51:00': [['brentford-vs-leeds-united',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900983/matchfacts/brentford-vs-leeds-united',
   'https://www.oddschecker.com/football/english/premier-league/brentford-v-leeds/winner'],
  ['chelsea-vs-west-ham-united',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900985/matchfacts/chelsea-vs-west-ham-united',
   'https://www.oddschecker.com/football/english/premier-league/chelsea-v-west-ham/winner'],
  ['newcastle-united-vs-crystal-palace',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900988/matchfacts/newcastle-united-vs-crystal-palace',
   'https://www.oddschecker.com/football/english/premier-league/newcastle-v-crystal-palace/winner'],
  ['nottingham-forest-vs-afc-bournemouth',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900989/matchfacts/nottingham-forest-vs-afc-bournemouth',
   'https://www.oddschecker.com/football/english/premier-league/nottingham-forest-v-bournemouth/winner']],
 '2022-09-03 10:52:00': [['tottenham-hotspur-vs-fulham',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900990/matchfacts/tottenham-hotspur-vs-fulham',
   'https://www.oddschecker.com/football/english/premier-league/tottenham-v-fulham/winner'],
  ['wolverhampton-wanderers-vs-southampton',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900991/matchfacts/wolverhampton-wanderers-vs-southampton',
   'https://www.oddschecker.com/football/english/premier-league/wolverhampton-v-southampton/winner'],
  ['blackburn-rovers-vs-bristol-city',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915347/matchfacts/blackburn-rovers-vs-bristol-city',
   'https://www.oddschecker.com/football/english/championship/blackburn-v-bristol-city/winner'],
  ['luton-town-vs-wigan-athletic',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915335/matchfacts/luton-town-vs-wigan-athletic',
   'https://www.oddschecker.com/football/english/championship/luton-v-wigan/winner']],
 '2022-09-03 10:53:00': [['millwall-vs-cardiff-city',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915482/matchfacts/millwall-vs-cardiff-city',
   'https://www.oddschecker.com/football/english/championship/millwall-v-cardiff/winner'],
  ['norwich-city-vs-coventry-city',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915337/matchfacts/norwich-city-vs-coventry-city',
   'https://www.oddschecker.com/football/english/championship/norwich-v-coventry/winner'],
  ['preston-north-end-vs-birmingham-city',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915338/matchfacts/preston-north-end-vs-birmingham-city',
   'https://www.oddschecker.com/football/english/championship/preston-v-birmingham/winner'],
  ['rotherham-united-vs-watford',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915340/matchfacts/rotherham-united-vs-watford',
   'https://www.oddschecker.com/football/english/championship/rotherham-v-watford/winner']],
 '2022-09-03 10:54:00': [['swansea-city-vs-queens-park-rangers',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915481/matchfacts/swansea-city-vs-queens-park-rangers',
   'https://www.oddschecker.com/football/english/championship/swansea-v-qpr/winner'],
  ['genk-vs-st.truiden',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916972/matchfacts/genk-vs-st.truiden',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/genk-v-sint-truidense/winner'],
  ['hibernian-vs-kilmarnock',
   'SCOTLAND - PREMIERSHIP',
   'https://www.fotmob.com/match/3902850/matchfacts/hibernian-vs-kilmarnock',
   'https://www.oddschecker.com/football/scottish/premiership/hibernian-v-kilmarnock/winner'],
  ['livingston-vs-hearts',
   'SCOTLAND - PREMIERSHIP',
   'https://www.fotmob.com/match/3902851/matchfacts/livingston-vs-hearts',
   'https://www.oddschecker.com/football/scottish/premiership/livingston-v-hearts/winner']],
 '2022-09-03 10:55:00': [['motherwell-vs-dundee-united',
   'SCOTLAND - PREMIERSHIP',
   'https://www.fotmob.com/match/3902852/matchfacts/motherwell-vs-dundee-united',
   'https://www.oddschecker.com/football/scottish/premiership/motherwell-v-dundee-utd/winner'],
  ['ross-county-vs-aberdeen',
   'SCOTLAND - PREMIERSHIP',
   'https://www.fotmob.com/match/3902853/matchfacts/ross-county-vs-aberdeen',
   'https://www.oddschecker.com/football/scottish/premiership/ross-county-v-aberdeen/winner'],
  ['st.-johnstone-vs-st.-mirren',
   'SCOTLAND - PREMIERSHIP',
   'https://www.fotmob.com/match/3902854/matchfacts/st.-johnstone-vs-st.-mirren',
   'https://www.oddschecker.com/football/scottish/premiership/st-johnstone-v-st-mirren/winner']],
 '2022-09-03 11:08:00': [['real-madrid-vs-real-betis',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917962/matchfacts/real-madrid-vs-real-betis',
   'https://www.oddschecker.com/football/spain/la-liga-primera/real-madrid-v-real-betis/winner']],
 '2022-09-03 11:23:00': [['ajax-vs-cambuur',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900378/matchfacts/ajax-vs-cambuur',
   'https://www.oddschecker.com/football/netherlands/eredivisie/ajax-v-cambuur-leeuwarden/winner'],
  ['braga-vs-vitoria-de-guimaraes',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937424/matchfacts/braga-vs-vitoria-de-guimaraes',
   'https://www.oddschecker.com/football/portugal/primeira-liga/braga-v-guimaraes/winner']],
 '2022-09-03 11:53:00': [['auxerre-vs-marseille',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904491/matchfacts/auxerre-vs-marseille',
   'https://www.oddschecker.com/football/france/ligue-1/auxerre-v-marseille/winner'],
  ['austria-wien-vs-austria-lustenau',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917285/matchfacts/austria-wien-vs-austria-lustenau',
   'https://www.oddschecker.com/football/austria/bundesliga/fk-austria-vienna-v-austria-lustenau/winner'],
  ['salzburg-vs-wsg-tirol',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917286/matchfacts/salzburg-vs-wsg-tirol',
   'https://www.oddschecker.com/football/austria/bundesliga/fc-salzburg-v-wsg-swarovski-tirol/winner'],
  ['sturm-graz-vs-hartberg',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917274/matchfacts/sturm-graz-vs-hartberg',
   'https://www.oddschecker.com/football/austria/bundesliga/sk-sturm-graz-v-hartberg/winner']],
 '2022-09-03 12:53:00': [['milan-vs-inter',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919123/matchfacts/milan-vs-inter',
   'https://www.oddschecker.com/football/italy/serie-a/ac-milan-v-inter-milan/winner'],
  ['sion-vs-basel',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905443/matchfacts/sion-vs-basel',
   'https://www.oddschecker.com/football/switzerland/super-league/fc-sion-v-basel/winner']],
 '2022-09-03 13:08:00': [['eupen-vs-kortrijk',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916977/matchfacts/eupen-vs-kortrijk',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/eupen-v-kv-kortrijk/winner'],
  ['kv-mechelen-vs-rfc-seraing',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916975/matchfacts/kv-mechelen-vs-rfc-seraing',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/kv-mechelen-v-rfc-seraing/winner']],
 '2022-09-03 13:23:00': [['aston-villa-vs-manchester-city',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900982/matchfacts/aston-villa-vs-manchester-city',
   'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-man-city/winner'],
  ['real-sociedad-vs-atletico-madrid',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917960/matchfacts/real-sociedad-vs-atletico-madrid',
   'https://www.oddschecker.com/football/spain/la-liga-primera/real-sociedad-v-atletico-madrid/winner'],
  ['eintracht-frankfurt-vs-rb-leipzig',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903584/matchfacts/eintracht-frankfurt-vs-rb-leipzig',
   'https://www.oddschecker.com/football/germany/bundesliga/eintracht-frankfurt-v-rb-leipzig/winner']],
 '2022-09-03 13:38:00': [['fc-twente-vs-psv-eindhoven',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900379/matchfacts/fc-twente-vs-psv-eindhoven',
   'https://www.oddschecker.com/football/netherlands/eredivisie/fc-twente-v-psv/winner']],
 '2022-09-03 13:53:00': [['lyon-vs-angers',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904496/matchfacts/lyon-vs-angers',
   'https://www.oddschecker.com/football/france/ligue-1/lyon-v-angers/winner']],
 '2022-09-03 14:53:00': [['sparta-rotterdam-vs-fc-volendam',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900380/matchfacts/sparta-rotterdam-vs-fc-volendam',
   'https://www.oddschecker.com/football/netherlands/eredivisie/sparta-rotterdam-v-fc-volendam/winner']],
 '2022-09-03 15:23:00': [['fc-z%C3%BCrich-vs-lugano',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905442/matchfacts/fc-z%C3%BCrich-vs-lugano',
   'https://www.oddschecker.com/football/switzerland/super-league/fc-zurich-v-lugano/winner']],
 '2022-09-03 15:38:00': [['lazio-vs-napoli',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919122/matchfacts/lazio-vs-napoli',
   'https://www.oddschecker.com/football/italy/serie-a/lazio-v-napoli/winner'],
  ['standard-liege-vs-oostende',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916971/matchfacts/standard-liege-vs-oostende',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/standard-liege-v-kv-oostende/winner']],
 '2022-09-03 15:53:00': [['sevilla-vs-barcelona',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917961/matchfacts/sevilla-vs-barcelona',
   'https://www.oddschecker.com/football/spain/la-liga-primera/sevilla-v-barcelona/winner'],
  ['nantes-vs-paris-saint-germain',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904493/matchfacts/nantes-vs-paris-saint-germain',
   'https://www.oddschecker.com/football/france/ligue-1/nantes-v-psg/winner'],
  ['go-ahead-eagles-vs-feyenoord',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900381/matchfacts/go-ahead-eagles-vs-feyenoord',
   'https://www.oddschecker.com/football/netherlands/eredivisie/go-ahead-eagles-v-feyenoord/winner']],
 '2022-09-03 16:23:00': [['minnesota-united-vs-fc-dallas',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787643/matchfacts/minnesota-united-vs-fc-dallas',
   'https://www.oddschecker.com/football/world/usa/mls/minnesota-united-v-fc-dallas/winner'],
  ['gil-vicente-vs-fc-porto',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937425/matchfacts/gil-vicente-vs-fc-porto',
   'https://www.oddschecker.com/football/portugal/primeira-liga/gil-vicente-v-fc-porto/winner']],
 '2022-09-03 18:23:00': [['columbus-crew-vs-chicago-fire-fc',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787654/matchfacts/columbus-crew-vs-chicago-fire-fc',
   'https://www.oddschecker.com/football/world/usa/mls/columbus-crew-v-chicago-fire/winner']],
 '2022-09-03 18:58:00': [['pachuca-vs-santos-laguna',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888625/matchfacts/pachuca-vs-santos-laguna',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/cf-pachuca-v-club-santos-laguna/winner']],
 '2022-09-03 19:53:00': [['new-york-red-bulls-vs-philadelphia-union',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787644/matchfacts/new-york-red-bulls-vs-philadelphia-union',
   'https://www.oddschecker.com/football/world/usa/mls/new-york-red-bulls-v-philadelphia-union/winner']],
 '2022-09-03 20:23:00': [['fc-cincinnati-vs-charlotte-fc',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787645/matchfacts/fc-cincinnati-vs-charlotte-fc',
   'https://www.oddschecker.com/football/world/usa/mls/fc-cincinnati-v-charlotte-fc/winner']],
 '2022-09-03 20:58:00': [['atlas-vs-club-universidad-nacional',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888627/matchfacts/atlas-vs-club-universidad-nacional',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/atlas-v-pumas-unam/winner'],
  ['monterrey-vs-mazatlan-fc',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888626/matchfacts/monterrey-vs-mazatlan-fc',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/cf-monterrey-v-mazatlan-fc/winner']],
 '2022-09-03 21:23:00': [['nashville-sc-vs-austin-fc',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787652/matchfacts/nashville-sc-vs-austin-fc',
   'https://www.oddschecker.com/football/world/usa/mls/nashville-sc-v-austin-fc/winner']],
 '2022-09-03 22:58:00': [['cf-america-vs-tigres',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888628/matchfacts/cf-america-vs-tigres',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/cf-america-v-tigres/winner']]}

In [19]:
c = days_data.copy()
todays_data = []

while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                r = executor.map(stats_odds, stats_urls, odds_urls)
            for result in r:
                todays_data.append(result)
            del c[clock]
    print(f'{len(c)} halftimes left -- {len(todays_data)}\n')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

open_file = open(f'{today}_data.pkl', "wb")
pickle.dump(todays_data, open_file)
open_file.close()

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

6 halftimes left -- 0

['https://www.fotmob.com/match/3919124/matchfacts/monza-v

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes left -- 7

1 halftimes

In [278]:
open_file = open('20220828_data.pkl', "rb")
xcxc = pickle.load(open_file)
open_file.close()

In [6]:
def predicted_probs(results):
    d = {}
    for k, v in results.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    for_pred = [[d[key] for key in pred_cols.values()]]
    pred = model[-1].predict_proba(for_pred)
    return pred

In [7]:
def make_bet(decimal_odds, predicted_prob, k_weight):
    if 1/decimal_odds >= predicted_prob:
        return 'no value'
    else:
        return kelly(predicted_prob, decimal_odds, k_weight)

In [8]:
def print_advice(results):
    try:
        d_h_a = ['draw'] + results['odds_url'].split('/')[-2].split('-v-')
        order_decimal_odds = [getname(odds_name, list(results['odds'].keys())) for odds_name in d_h_a]
        print(order_decimal_odds)
        p_probs = predicted_probs(results)[0]
        predicted = np.argmax(p_probs)
        decimal_odds = [results['odds'][key] for key in order_decimal_odds]
        print(decimal_odds)
        bet_made = {}
        print('\033[1m' + results['match_name'] + '\033[0m')
        print(f'Predicted winner is {predicted}\n')
        for p_prob, d_odd, win in zip(p_probs, decimal_odds, ['draw', 'home','away']):
            p_prob = round(p_prob, 3)
            d_odd = round(d_odd, 3)
            size = make_bet(d_odd, p_prob, 0.5)
            print(f'{win}: predicted prob: {p_prob*100}% - oddschecker: {round(1/d_odd*100,2)}% ({d_odd}) ---- bet {size}')
            bet_made[win] = size
        print('\n')
        bet_made['odds_checker_sum'] = np.sum([1/o for o in decimal_odds])
        bet_made['winner'] = predicted
        bet_made['predicted_probs'] = np.round(p_probs, 3)
        bets = {'bet':bet_made} 
        row = {**results, **bets}
    except:
        row = results
    return row

In [9]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [10]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [11]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(1)
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-lv1jm0-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print('not halftime just yet ... sleeping\n')
            time.sleep(3*60)
            ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        button.click()
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        print(score)
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
#         game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        match_name = url.split('/')[-1]
        game_stats['match_name'] = match_name
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    driver.quit()
    return game_stats

In [12]:
def get_stats(url):
    try:
        page = requests.get(url)
        time.sleep(random.randint(0, 4))
        soup = BeautifulSoup(page.content, 'html.parser')
        ht = soup.find("span", {"class": "css-lv1jm0-bottomRow"}).text
        if ht != 'Half-Time':
                print('not halftime just yet ... sleeping\n')
                time.sleep(3*60)
                ht = soup.find("span", {"class": "css-lv1jm0-bottomRow"}).text
        div1 = soup.find_all("li", {"class": "css-radwzz-Stat e683amr6"})
        all_stats = {k[1]: [k[0], k[2]] for k in [z.split('|') for z in [x for x in [d.get_text('|') for d in div1]
                  if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
        score = [score[0], score[1]]
        posession = soup.find("div", {"class": "css-7s52se-PossessionWheel e683amr3"}).get_text('|').split('|')
        posession = [posession[1], posession[3]]
        all_stats['posession'] = posession
        all_stats['score'] = score
        game_stats = {}
        for name, stats in all_stats.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
        match_name = url.split('/')[-1]
        game_stats['match_name'] = match_name
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    return game_stats

In [67]:
get_stats('https://www.fotmob.com/match/3900365/matchfacts/az-alkmaar-vs-nec-nijmegen')

not halftime just yet ... sleeping



{'Expected goals (xG)': ['2.62', '0.07'],
 'Total shots': ['30', '4'],
 'Big chances': ['4', '0'],
 'Big chances missed': ['4', '0'],
 'Accurate passes': ['534', '266'],
 'Accurate passes percentage': [0.89, 0.76],
 'Fouls committed': ['7', '10'],
 'Offsides': ['1', '0'],
 'Corners': ['8', '3'],
 'Shots off target': ['8', '2'],
 'Shots on target': ['13', '2'],
 'Blocked shots': ['9', '0'],
 'Hit woodwork': ['0', '0'],
 'Shots inside box': ['17', '1'],
 'Shots outside box': ['13', '3'],
 'xG first half': ['0.37', '0.04'],
 'xG second half': ['2.25', '0.03'],
 'xG open play': ['2.35', '0.04'],
 'xG set play': ['0.27', '0.03'],
 'xG on target (xGOT)': ['1.74', '0.89'],
 'Passes': ['600', '349'],
 'Own half': ['197', '199'],
 'Opposition half': ['337', '67'],
 'Accurate long balls': ['29', '23'],
 'Accurate long balls percentage': [0.59, 0.4],
 'Accurate crosses': ['6', '3'],
 'Accurate crosses percentage': [0.33, 0.33],
 'Throws': ['18', '13'],
 'Yellow cards': ['1', '2'],
 'Red cards': [

In [13]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership',
             'ITALY - SERIE A':'italy/serie-a'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-1f0alcw-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [14]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [15]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    r = {}
    test = scraper_data.copy()
    for date, data in test.copy().items():
        if len(data) > 5:
            x = [data[i:i + 4] for i in range(0, len(data), 4)]
            d = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            groups = len(x)
            times = [str(d + timedelta(minutes = i * 1)) for i in range(-groups//2+1,groups//2+1, 1)]
            new = dict(zip(times, x))
            del test[date]
            r = {**r, **new}
        else:
            r[date] = data
    scraper_data = dict(sorted(r.items(), key=lambda x: x[0]))
    return scraper_data

In [16]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'odds_url':url, 'odds': '', 'full_odds': ''}
        driver.implicitly_wait(1)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features="html.parser")
        odds_containers = soup.find_all("div", {"class": "oddsAreaWrapper_o17xb9rs RowLayout_refg9ta"})
        names = soup.find_all("div", {"class": "BetRowLeftBetContent_b2f00kt"})
        names = [n.text for n in names]
        odds_containers = [container.find_all('button') for container in odds_containers]
        odds = [[z.text for z in container] for container in odds_containers]
        odds = [[float(Fraction(j.replace(' ',''))) + 1.0 for j in o][:-3] for o in odds]
        median_odds = [statistics.median(o) for o in odds]
        median_odds = dict(zip(names, median_odds))
        odds = {'odds_url':url, 'odds': median_odds, 'full_odds': odds}
    except:
        return {'odds_url':url, 'odds': '', 'full_odds': ''}
    driver.quit()
    return odds

In [17]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(20)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    result = print_advice(row)
    return result